# Retrieval Evaluation

We'd like to compare several retrieval techniques to find the best for our use case, querying the kenya 2010 constitution.

The options are:
- lexical search
- semantic search
- a hybrid of lexical and semantic search

We'll use synthetic data obtained by generating questions for each of the constitution's articles using an LLM.

We'll run these questions through the retrieval process and use two metrics for evaluating the results:

- hit rate
- mean reciprocal rank

As a baseline, we'll use the barebones [minsearch][0] library which implements simple lexical search using [TF-IDF][1] for document representation.

[0]: https://github.com/alexeygrigorev/minsearch/blob/main/minsearch.py
[1]: https://en.wikipedia.org/wiki/Tf%E2%80%93idf

## Data preparation

In [1]:
pip install -q pandas tqdm


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

Load previously generated evaluation data:

In [3]:
![ -f rag_evaluation_data.csv ] || wget https://raw.githubusercontent.com/programmer-ke/katiba-chat/refs/heads/master/notebooks/rag_evaluation_data.csv

In [4]:
df_eval_data = pd.read_csv("rag_evaluation_data.csv")
df_eval_data

,question,article_number
0,Who holds all sovereign power in Kenya accordi...,1
1,How can the people of Kenya exercise their sov...,1
2,To which state organs is sovereign power deleg...,1
3,At which levels is the sovereign power of the ...,1
4,What makes this Constitution the ultimate auth...,2
...,...,...
1312,What was the previous constitution that was in...,264
1313,When did the previous constitution cease to be...,264
1314,What happened to the previous constitution on ...,264
1315,What is the Sixth Schedule in relation to the ...,264


It has two columns, a question and the associated article number. There are several questions for each article.

Next, we load the articles:

In [5]:
![ ! -f constitution.json ] && wget https://raw.githubusercontent.com/programmer-ke/constitution_kenya/refs/heads/master/json/ConstitutionKenya2010.json -O constitution.json

In [6]:
import json

with open('constitution.json', 'rt') as f:
    articles = json.load(f)

documents = []
for article in articles:
    
    article_text = "".join(article['lines'])
    article_title = f"Article {article['number']}: {article['title']}"
    chapter_number, chapter_title = article['chapter']
    chapter_text = f"Chapter {chapter_number}: {chapter_title}"
    part_text = ""
    
    if article['part']:
        part_num, part_title = article['part']
        part_text = f'Part {part_num}: {part_title}'
        
    documents.append({
        "title": article_title,
        "clauses": article_text,
        "chapter": chapter_text,
        "part": part_text,
        "number": article['number']
    })

In [7]:
documents[:2]

[{'title': 'Article 1: Sovereignty of the people.',
  'clauses': '(1)  All sovereign power belongs to the people of Kenya and shall be exercised\nonly in accordance with this Constitution.\n(2)  The people may exercise their sovereign power either directly or through their\ndemocratically elected representatives.\n(3)  Sovereign power under this Constitution is delegated to the following State\norgans, which shall perform their functions in accordance with this Constitution—\n(a) Parliament and the legislative assemblies in the county governments;\n(b) the national executive and the executive structures in the county\ngovernments; and\n(c) the Judiciary and independent tribunals.\n(4)  The sovereign power of the people is exercised at—\n(a) the national level; and\n(b) the county level.\n',
  'chapter': 'Chapter 1: SOVEREIGNTY OF THE PEOPLE AND SUPREMACY OF THIS CONSTITUTION',
  'part': '',
  'number': 1},
 {'title': 'Article 2: Supremacy of this Constitution.',
  'clauses': '(1)  This

In [8]:
evaluation_data = df_eval_data.to_dict(orient='records')
evaluation_data[:3]

[{'question': 'Who holds all sovereign power in Kenya according to this Constitution?',
  'article_number': 1},
 {'question': 'How can the people of Kenya exercise their sovereign power?',
  'article_number': 1},
 {'question': 'To which state organs is sovereign power delegated?',
  'article_number': 1}]

## Baseline with minsearch

Get minsearch:

In [9]:
![ ! -f minsearch.py ] && wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py -O minsearch.py

Next is to index the documents for search:

In [10]:
import minsearch

index = minsearch.Index(text_fields=['title', 'clauses', 'chapter', 'part'], keyword_fields=[])
index.fit(documents)

In [11]:
def search(query):
    return index.search(query, num_results=5)

Test search with the first question:

In [12]:
search(evaluation_data[0]['question'])

[{'title': 'Article 2: Supremacy of this Constitution.',
  'clauses': '(1)  This Constitution is the supreme law of the Republic and binds all persons\nand all State organs at both levels of government.\n(2)  No person may claim or exercise State authority except as authorised under\nthis Constitution.\n(3)  The validity or legality of this Constitution is not subject to challenge by or\nbefore any court or other State organ.\n(4)  Any law, including customary law, that is inconsistent with this Constitution\nis void to the extent of the inconsistency, and any act or omission in contravention\nof this Constitution is invalid.\n(5)  The general rules of international law shall form part of the law of Kenya.\n(6)  Any treaty or convention ratified by Kenya shall form part of the law of Kenya\nunder this Constitution.\n',
  'chapter': 'Chapter 1: SOVEREIGNTY OF THE PEOPLE AND SUPREMACY OF THIS CONSTITUTION',
  'part': '',
  'number': 2},
 {'title': 'Article 255: Amendment of this Constitu

We'll determine the relevance of each set of results by indicating which of the results is the correct answer as per the evaluation data:

In [13]:
from tqdm import tqdm
relevance = []

for q in tqdm(evaluation_data):
    results = search(q['question'])
    hits = [result['number'] == q['article_number'] for result in results]
    relevance.append(hits)

100%|██████████████████████████████████████████████████████████████| 1317/1317 [00:21<00:00, 60.83it/s]


The results for the first few questions, showing where the expected response is in the 5 responses:

In [14]:
relevance[:7]

[[False, False, False, True, False],
 [True, False, False, False, False],
 [False, False, True, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [True, False, False, False, False]]

### Hit Rate

The hit rate metric will be defined as any time the correct answer to the question is found within the first five results

In [15]:
def hit_rate(relevance):
    target_found = [any(results) for results in relevance]
    hit_count = sum(target_found)
    return hit_count / len(relevance)

In [16]:
hit_rate(relevance[:7])

0.8571428571428571

There's an 86% hit rate for the first 5 questions.

### Mean Reciprocal Rank

MRR shows how highly ranked the expected response it. The higher it is ranked in the set of results, the greater the score.

The reciprocal rank is calculcated based on the position of the correct response in the set of results, as 1/N where N is the position. First position would result in 1, second 1/2, third 1/3 and so on. This is then averaged over all the questions.

In [17]:
def rr(question_results):
    for i, hit in enumerate(question_results):
        if hit:
            return 1/(i + 1)
    return 0

def mrr(relevance):
    scores = [rr(hits) for hits in relevance]
    total = sum(scores)
    return total / len(relevance)

Calculating MRR for the first few results:

In [18]:
mrr(relevance[:7])

0.6547619047619048

Calculating the two metrics over all questions:

In [19]:
def score(relevance):
    hit_rate_score = hit_rate(relevance)
    mrr_score = mrr(relevance)
    avg_score = (hit_rate_score + mrr_score) / 2
    return {'hit_rate': hit_rate_score, 'mrr': mrr_score, 'avg_score': avg_score}

In [20]:
score(relevance)

{'hit_rate': 0.5535307517084282,
 'mrr': 0.41580612503163755,
 'avg_score': 0.48466843837003293}

## Lexical Search - Whoosh

[Whoosh][5] is a pure python embeddable search library with many useful features, striking a balance between the barebones minsearch library used previously and heavyweight search solutions like elastic search.

Install it:

[5]: https://github.com/Sygil-Dev/whoosh-reloaded

In [21]:
!pip install -q whoosh-reloaded


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Create an index:

In [22]:
from pathlib import Path

from whoosh import fields as F
from whoosh import index
from whoosh import qparser

schema = F.Schema(
    title=F.TEXT(stored=True),
    clauses=F.TEXT(stored=True),
    chapter=F.TEXT(stored=True),
    part=F.TEXT(stored=True),
    number=F.STORED,
)

index_dirname = "whoosh_index"
p = Path(index_dirname)
if not p.exists():
    p.mkdir()

doc_index = index.create_in(index_dirname, schema)
writer = doc_index.writer()
for doc in tqdm(documents):
    writer.add_document(**doc)
writer.commit()

100%|███████████████████████████████████████████████████████████████| 264/264 [00:00<00:00, 338.28it/s]


Create a search function:

In [23]:
def whoosh_search(question):
    with doc_index.searcher() as searcher:
        parser = qparser.MultifieldParser(['title', 'clauses', 'chapter', 'part'], schema=schema, group=qparser.OrGroup)
        query = parser.parse(question)
        results = searcher.search(query, limit=5)
        results = [dict(r) for r in results]
    return results

In [24]:
sample_question = evaluation_data[0]['question']
sample_question

'Who holds all sovereign power in Kenya according to this Constitution?'

In [25]:
whoosh_search(sample_question)

[{'chapter': 'Chapter 1: SOVEREIGNTY OF THE PEOPLE AND SUPREMACY OF THIS CONSTITUTION',
  'clauses': '(1)  All sovereign power belongs to the people of Kenya and shall be exercised\nonly in accordance with this Constitution.\n(2)  The people may exercise their sovereign power either directly or through their\ndemocratically elected representatives.\n(3)  Sovereign power under this Constitution is delegated to the following State\norgans, which shall perform their functions in accordance with this Constitution—\n(a) Parliament and the legislative assemblies in the county governments;\n(b) the national executive and the executive structures in the county\ngovernments; and\n(c) the Judiciary and independent tribunals.\n(4)  The sovereign power of the people is exercised at—\n(a) the national level; and\n(b) the county level.\n',
  'number': 1,
  'part': '',
  'title': 'Article 1: Sovereignty of the people.'},
 {'chapter': 'Chapter 14: NATIONAL SECURITY',
  'clauses': '(1)  There are estab

Next step is to determine relevance and calculate the evaluation metrics:

In [26]:
def get_relevance(evaluation_data, searcher):
    relevance = []
    for q in tqdm(evaluation_data):
        results = searcher(q['question'])
        hits = [result['number'] == q['article_number'] for result in results]
        relevance.append(hits)
    return relevance

In [27]:
relevance = get_relevance(evaluation_data, whoosh_search)

100%|██████████████████████████████████████████████████████████████| 1317/1317 [00:33<00:00, 39.39it/s]


In [28]:
score(relevance)

{'hit_rate': 0.8116932422171602,
 'mrr': 0.6781700835231587,
 'avg_score': 0.7449316628701594}

Using a production ready library has improved our score for lexical search.

## Semantic Search - Sentence Transformers

[Sentence transformers][8] is a python library that provides similarity search functionality based on BERT embeddings.

[8]: https://sbert.net/

Installation:

In [29]:
!pip install -q sentence-transformers --extra-index-url https://download.pytorch.org/whl/cpu

We'll use a model from trained on millions of question-answer pairs and optimized for a good balance between speed and accuracy.

https://sbert.net/docs/sentence_transformer/pretrained_models.html#multi-qa-models

In [29]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

/home/krm/projects/katiba-chat/notebooks/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Transform the documents and query into embeddings:

In [30]:
texts = [
    """
    chapter: {chapter}
    part: {part}
    title: {title}
    clauses: {clauses}
    """.strip().format(**doc) for doc in documents
]

In [31]:
print(texts[20])

chapter: Chapter 4: THE BILL OF RIGHTS
    part: Part 1: GENERAL PROVISIONS TO THE BILL OF RIGHTS
    title: Article 21: Implementation of rights and fundamental freedoms.
    clauses: (1)  It is a fundamental duty of the State and every State organ to observe,
respect, protect, promote and fulfil the rights and fundamental freedoms in the Bill
of Rights.
(2)  The State shall take legislative, policy and other measures, including the
setting of standards, to achieve the progressive realisation of the rights guaranteed
under Article 43.
(3)  All State organs and all public officers have the duty to address the needs
of vulnerable groups within society, including women, older members of society,
persons with disabilities, children, youth, members of minority or marginalised
communities, and members of particular ethnic, religious or cultural communities.
(4)  The State shall enact and implement legislation to fulfil its international
obligations in respect of human rights and fundamental

In [32]:
document_embeddings = model.encode(texts)

Then create and test the search function:

In [33]:
def sentence_transformers_search(query):
    query_embeddings = model.encode(query)
    results = util.semantic_search(query_embeddings, document_embeddings, top_k=5)
    indices = [r['corpus_id'] for r in results[0]]
    matching_docs = [documents[idx] for idx in indices]
    return matching_docs

In [34]:
query = evaluation_data[0]['question']; query

'Who holds all sovereign power in Kenya according to this Constitution?'

In [35]:
sentence_transformers_search(query)

[{'title': 'Article 1: Sovereignty of the people.',
  'clauses': '(1)  All sovereign power belongs to the people of Kenya and shall be exercised\nonly in accordance with this Constitution.\n(2)  The people may exercise their sovereign power either directly or through their\ndemocratically elected representatives.\n(3)  Sovereign power under this Constitution is delegated to the following State\norgans, which shall perform their functions in accordance with this Constitution—\n(a) Parliament and the legislative assemblies in the county governments;\n(b) the national executive and the executive structures in the county\ngovernments; and\n(c) the Judiciary and independent tribunals.\n(4)  The sovereign power of the people is exercised at—\n(a) the national level; and\n(b) the county level.\n',
  'chapter': 'Chapter 1: SOVEREIGNTY OF THE PEOPLE AND SUPREMACY OF THIS CONSTITUTION',
  'part': '',
  'number': 1},
 {'title': 'Article 2: Supremacy of this Constitution.',
  'clauses': '(1)  This

Next, we evaluate this implementation:

In [36]:
score(get_relevance(evaluation_data, sentence_transformers_search))

100%|██████████████████████████████████████████████████████████████| 1317/1317 [00:58<00:00, 22.58it/s]


{'hit_rate': 0.8587699316628702,
 'mrr': 0.7339281194634271,
 'avg_score': 0.7963490255631487}

This shows an improved score over lexical search.

## Hybrid Retrieval

With hybrid retrieval, we'll use a combination of lexical and semantic search and re-rank the results based on a combined score of both approaches.

The technique is known as **Reciprocal Rank Fusion (RRF)**. For each document in the results, we calculate a score based on what position they rank for a given query, then add up the scores for the two retrieval approaches.

The score for a particular document in a specific retrieval is calculated by the formula: _1/(k + r(d))_, where _r(d)_ is the rank of the document in the retrieval results. The higher it ranks, the larger the score. _k_ is a constant that is typically set to 60. Some properties of this formula are explained [here][9].

[9]: https://medium.com/@devalshah1619/mathematical-intuition-behind-reciprocal-rank-fusion-rrf-explained-in-2-mins-002df0cc5e2a

Implementing the rrf score:

In [37]:
def rrf_score(rank, k=60):
    return 1 / (k + rank)

The score decreases with the rank:

In [38]:
[rrf_score(i) for i in range(1, 6)]

[0.01639344262295082,
 0.016129032258064516,
 0.015873015873015872,
 0.015625,
 0.015384615384615385]

Combining the scores:

In [39]:
def rrf(ranked_results, scores):
    for i, result in enumerate(ranked_results):
        doc_id = result['number']
        scores[doc_id] = rrf_score(i + 1) + scores.get(doc_id, 0)

We implement a combined search and rerank the documents using RRF:

In [40]:
def hybrid_search(query, lexical_searcher, semantic_searcher):
    lexical_search_results = lexical_searcher(query)
    semantic_search_results = semantic_searcher(query)

    combined_scores = {}
    rrf(lexical_search_results, combined_scores)
    rrf(semantic_search_results, combined_scores)

    # re-rank the results based on score
    reranked_scores = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    final_results = []
    for doc_id, score in reranked_scores[:5]:
        # doc id is article number, we can access article by index
        final_results.append(documents[doc_id - 1])
    return final_results        

### Hybrid Retrieval - Whoosh and Sentence Transformers

In [41]:
q = evaluation_data[0]['question']; q

'Who holds all sovereign power in Kenya according to this Constitution?'

In [43]:
hybrid_search(q, whoosh_search, sentence_transformers_search)

[{'title': 'Article 1: Sovereignty of the people.',
  'clauses': '(1)  All sovereign power belongs to the people of Kenya and shall be exercised\nonly in accordance with this Constitution.\n(2)  The people may exercise their sovereign power either directly or through their\ndemocratically elected representatives.\n(3)  Sovereign power under this Constitution is delegated to the following State\norgans, which shall perform their functions in accordance with this Constitution—\n(a) Parliament and the legislative assemblies in the county governments;\n(b) the national executive and the executive structures in the county\ngovernments; and\n(c) the Judiciary and independent tribunals.\n(4)  The sovereign power of the people is exercised at—\n(a) the national level; and\n(b) the county level.\n',
  'chapter': 'Chapter 1: SOVEREIGNTY OF THE PEOPLE AND SUPREMACY OF THIS CONSTITUTION',
  'part': '',
  'number': 1},
 {'title': 'Article 4: Declaration of the Republic.',
  'clauses': '(1)  Kenya i

Evaluate it:

In [44]:
hybrid_searcher_st = lambda query: hybrid_search(query, whoosh_search, sentence_transformers_search)

score(get_relevance(evaluation_data, hybrid_searcher_st))

100%|██████████████████████████████████████████████████████████████| 1317/1317 [01:30<00:00, 14.57it/s]


{'hit_rate': 0.8990129081245254,
 'mrr': 0.743343457352569,
 'avg_score': 0.8211781827385471}

Hybrid retrieval gives the best score, showing that both lexical and semantic retrieval complement each other in useful ways.

## Conclusion

We used synthetic data generated by an LLM to evaluate various retrieval techniques: lexical, semantic and a hybrid of both.

For evaluation, we used two metrics: **hit rate**; the proportion of evaluation results where the expected answer is within the results, and **mean reciprocal rank** (MRR); how highly ranked the expected answer was on average.

Hybrid ranked highest 

While this indicates a good starting point in implementing a retrieval system, the evaluation data is synthetic data, so it may differ from real world data. As such, real world data may be collected and used as ground truth to more accurately evaluate and improve the retrieval process.